CREAMOS LAS MÉTRICA PARA EL ANÁLISIS DE VENTAS.

De las tablas ventas podemos determinar:
-los valores totales de cada venta
-la fecha de la primera y última compra
-Compras por cliente.
-Actividad por día, mes y año.

In [1]:
#importamos librerias
import pandas as pd    # Librería para tablas y tratamiento de datos.
#Para una visualización completa de todas las columnas:
pd.set_option('display.max_columns', None)
#Accedemos a las funciones en soporte_dts de la carpeta src
from src import soporte_dts as sdts
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
df_ventas_cliente_prod=sdts.leer_datos('clientes_productos_ventas_limpio.csv')

In [3]:
df_ventas_cliente_prod.columns

Index(['id_venta', 'id_producto', 'cantidad', 'fecha', 'id_cliente', 'nombre',
       'apellido', 'ciudad', 'precio', 'categoria', 'descripcion', 'pais',
       'valor_total_venta'],
      dtype='object')

In [4]:
#Añadimos columna para determinar el dia de la semana que se realizan las compras:
df_ventas_cliente_prod['fecha']=pd.to_datetime(df_ventas_cliente_prod['fecha'])

In [5]:
df_ventas_cliente_prod['dia_semana_venta']=df_ventas_cliente_prod['fecha'].dt.dayofweek

In [6]:
#añadimos una nueva columna donde calculamos el valor de cada venta:
df_ventas_cliente_prod['valor_total_venta']=df_ventas_cliente_prod['cantidad']*df_ventas_cliente_prod['precio']
df_ventas_cliente_prod.sample(2)

,id_venta,id_producto,cantidad,fecha,id_cliente,nombre,apellido,ciudad,precio,categoria,descripcion,pais,valor_total_venta,dia_semana_venta
63814,V003665,P0554,1,2024-04-28,C0224,Laura,García,Madrid,42.83,Clothing,TteUZNrAl3eMlXFKxQwANOsFAYtLgZlpmzs16dWxnqHsUI...,España,42.83,6
64944,V000510,P0200,3,2021-10-21,C0042,María,Pérez,Madrid,6.67,Home & Kitchen,hSrOMW6t6yrKnapDLYDVKXsiE4b759i5h8GjGTTmXsSXyN...,España,20.01,3


In [7]:
#Procedemos a guardar el df _limpio:
df_ventas_cliente_prod.to_csv('clientes_productos_ventas_limpio.csv',index=False)

In [8]:
# agruparemos la columna clientes y determinaremos lafecha asociada minima y máxima, el numero de ventas registras, y el 
#Creamos un Dataframe con las métricas que podemos extraer de los datos:
#aplicamos diferentes funcioones de agregación por columna:
metricas_clientes=df_ventas_cliente_prod.groupby('id_cliente', as_index=False).agg({'fecha':['min','max'],'id_venta':'nunique','valor_total_venta':'sum'})
metricas_clientes.columns=['id_cliente','primera_compra','ultima_compra','numero_compras','valor_total']


In [9]:
metricas_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_cliente      210 non-null    object        
 1   primera_compra  210 non-null    datetime64[ns]
 2   ultima_compra   210 non-null    datetime64[ns]
 3   numero_compras  210 non-null    int64         
 4   valor_total     210 non-null    float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(1)
memory usage: 8.3+ KB


In [10]:
#Obtenemos informacion adicional a partir de las fechas: para ello debemos modificar el tiopo de dat object a datatime:


metricas_clientes['primera_compra']=pd.to_datetime(metricas_clientes['primera_compra'])
metricas_clientes['ultima_compra']=pd.to_datetime(metricas_clientes['ultima_compra'])

In [11]:
#Creamos columnas con año dia y fecha de cada transacción:
metricas_clientes['dia_primera_compra']=metricas_clientes['primera_compra'].dt.day
metricas_clientes['mes_primera_compra']=metricas_clientes['primera_compra'].dt.month
metricas_clientes['año_primera_compra']=metricas_clientes['primera_compra'].dt.year

metricas_clientes['dia_ultima_compra']=metricas_clientes['ultima_compra'].dt.day
metricas_clientes['mes_ultima_compra']=metricas_clientes['ultima_compra'].dt.month
metricas_clientes['año_ultima_compra']=metricas_clientes['ultima_compra'].dt.year

In [12]:
metricas_clientes.sample(5)


,id_cliente,primera_compra,ultima_compra,numero_compras,valor_total,dia_primera_compra,mes_primera_compra,año_primera_compra,dia_ultima_compra,mes_ultima_compra,año_ultima_compra
206,C0295,2020-02-04,2024-07-03,24,61059.30,4,2,2020,3,7,2024
202,C0285,2020-01-11,2024-03-29,39,109046.80,11,1,2020,29,3,2024
95,C0142,2020-02-01,2024-07-04,35,95321.43,1,2,2020,4,7,2024
139,C0203,2020-01-13,2024-06-05,38,107605.43,13,1,2020,5,6,2024
28,C0043,2020-01-06,2024-05-16,42,115738.11,6,1,2020,16,5,2024


In [13]:
#Calculamos la antiguedad en días con respecto a la fecha actual
metricas_clientes['antiguedad_dias']=(datetime.today()-metricas_clientes['primera_compra']).dt.days



In [14]:
#Calculamos el tiempo de actividad y la frecuencia( nºcompras/tiempo de actividad)
metricas_clientes['actividad_dias']=(metricas_clientes['ultima_compra']-metricas_clientes['primera_compra']).dt.days
metricas_clientes['actividad_meses']=metricas_clientes['actividad_dias'].apply(lambda x: round(x/30),1)
metricas_clientes['actividad_años']=metricas_clientes['actividad_meses'].apply(lambda x: round(x/12),1)
metricas_clientes['compras_por_mes']=metricas_clientes['numero_compras']/metricas_clientes['actividad_meses']
metricas_clientes['compras_por_año']=metricas_clientes['numero_compras']/metricas_clientes['actividad_años']

C:\Users\marg_\AppData\Local\Temp\ipykernel_21172\2051430706.py:3: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  metricas_clientes['actividad_meses']=metricas_clientes['actividad_dias'].apply(lambda x: round(x/30),1)
C:\Users\marg_\AppData\Local\Temp\ipykernel_21172\2051430706.py:4: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  metricas_clientes['actividad_años']=metricas_clientes['actividad_meses'].apply(lambda x: round(x/12),1)


In [15]:
metricas_clientes.sample(5)


,id_cliente,primera_compra,ultima_compra,numero_compras,valor_total,dia_primera_compra,mes_primera_compra,año_primera_compra,dia_ultima_compra,mes_ultima_compra,año_ultima_compra,antiguedad_dias,actividad_dias,actividad_meses,actividad_años,compras_por_mes,compras_por_año
103,C0152,2020-02-11,2024-05-27,35,86949.16,11,2,2020,27,5,2024,1800,1567,52,4,0.673077,8.75
2,C0003,2020-01-26,2024-06-09,38,92739.58,26,1,2020,9,6,2024,1816,1596,53,4,0.716981,9.50
79,C0120,2020-01-23,2024-05-14,26,67295.55,23,1,2020,14,5,2024,1819,1573,52,4,0.500000,6.50
36,C0051,2020-02-07,2024-03-24,34,86532.97,7,2,2020,24,3,2024,1804,1507,50,4,0.680000,8.50
193,C0271,2020-06-27,2024-06-11,36,91912.95,27,6,2020,11,6,2024,1663,1445,48,4,0.750000,9.00


In [16]:
metricas_clientes.to_csv('csv_limpios_finales\metricas_cliente.csv')

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\marg_\AppData\Local\Temp\ipykernel_21172\1741905565.py:1: SyntaxWarning: invalid escape sequence '\m'
  metricas_clientes.to_csv('csv_limpios_finales\metricas_cliente.csv')


Separamos las columnas correspondientes a información clientes para tenerla en separado:

In [17]:
df_ventas_cliente_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79920 entries, 0 to 79919
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_venta           79920 non-null  object        
 1   id_producto        79920 non-null  object        
 2   cantidad           79920 non-null  int64         
 3   fecha              79920 non-null  datetime64[ns]
 4   id_cliente         79920 non-null  object        
 5   nombre             79920 non-null  object        
 6   apellido           79920 non-null  object        
 7   ciudad             79920 non-null  object        
 8   precio             79920 non-null  float64       
 9   categoria          79920 non-null  object        
 10  descripcion        79920 non-null  object        
 11  pais               79920 non-null  object        
 12  valor_total_venta  79920 non-null  float64       
 13  dia_semana_venta   79920 non-null  int32         
dtypes: dat

In [18]:
df_clientes_informacion=df_ventas_cliente_prod[['id_cliente','nombre','apellido','ciudad','pais']]
df_clientes_informacion.to_csv('csv_limpios_finales\clientes_información.csv',index=False)

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\marg_\AppData\Local\Temp\ipykernel_21172\1031945816.py:2: SyntaxWarning: invalid escape sequence '\c'
  df_clientes_informacion.to_csv('csv_limpios_finales\clientes_información.csv',index=False)


In [19]:
df_productos_informacion=df_ventas_cliente_prod[['id_producto','precio','categoria','descripcion']]
df_ventas_informacion=df_ventas_cliente_prod[['id_venta','id_producto','id_cliente','cantidad','fecha','valor_total_venta']]
df_productos_informacion.to_csv('csv_limpios_finales\productos_informacion.csv',index=False)
df_ventas_informacion.to_csv('csv_limpios_finales\ ventas_informacion.csv',index=False)

<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\ '
C:\Users\marg_\AppData\Local\Temp\ipykernel_21172\3725780950.py:3: SyntaxWarning: invalid escape sequence '\p'
  df_productos_informacion.to_csv('csv_limpios_finales\productos_informacion.csv',index=False)
C:\Users\marg_\AppData\Local\Temp\ipykernel_21172\3725780950.py:4: SyntaxWarning: invalid escape sequence '\ '
  df_ventas_informacion.to_csv('csv_limpios_finales\ ventas_informacion.csv',index=False)
